In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
LABELS = ["Normal", "Fraud"]


In [ ]:
data = pd.read_csv('../input/credit-card-fraud-detection/creditcard.csv',sep=',')
data.head(10)


In [ ]:
data.isnull().values.any()

In [ ]:
fraud = data[data['Class']==1]

normal = data[data['Class']==0]

In [ ]:


print(fraud.shape,normal.shape)



In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

f.suptitle('Frequency of fraudulent and non-fraudulent transactions against the amount')
bins = 30
ax1.hist(fraud.Amount, bins = bins)

ax1.set_title('Fraudulent')
ax2.hist(normal.Amount, bins = bins)


ax2.set_title('Normal')
plt.xlabel('Amount ($)')
plt.ylabel('Number of Transactions')
plt.xlim((0, 20000))
plt.yscale('log')
plt.show();

In [ ]:
 plt.boxplot(data.Amount, labels = ['Amount'])

In [ ]:
import seaborn as sns #needed import for heatmap setting 

correlation = data.corr()
corr_features = correlation.index
plt.figure(figsize=(25,25))
g=sns.heatmap(data[corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
plt.scatter(data.V2, data.Amount) #scatter chart 

plt.xlabel('V2')

plt.ylabel('Amount ($)')

sns.regplot(data['V2'],data['Amount'])

# **OUTLIER DETECTION MODEL**

In [ ]:
Fraud = data[data['Class']==1]

# Declaration 
Valid = data[data['Class']==0]


outlier_fraction = len(Fraud)/float(len(Valid))

In [ ]:
print(outlier_fraction)

print("FRAUD CASES : {}".format(len(Fraud)))

print("VALID CASES : {}".format(len(Valid)))

In [ ]:
columns = data.columns.tolist()

# Filtering

columns = [c for c in columns if c not in ["Class"]]
# Storae of prediction var

target = "Class"
 
state = np.random.RandomState(42)
X = data[columns]
Y = data[target]
X_outliers = state.uniform(low=0, high=1, size=(X.shape[0], X.shape[1]))


#PRINT
print(X.shape)
print(Y.shape)

In [ ]:
classifiers = {
    "Isolation Forest":IsolationForest(n_estimators=100, max_samples=len(X), 
                                       
    contamination=outlier_fraction,random_state=state, verbose=0),
    
    "Local Outlier Factor":LocalOutlierFactor(n_neighbors=20, algorithm='auto', 
     leaf_size=30, metric='minkowski',
                                              
     p=2, metric_params=None, contamination=outlier_fraction),
   
}

In [ ]:
n_outliers = len(Fraud)
for i, (clf_name,clf) in enumerate(classifiers.items()):
    
    if clf_name == "Local Outlier Factor":
        y_pred = clf.fit_predict(X)
        scores_prediction = clf.negative_outlier_factor_
        
        
    elif clf_name == "Support Vector MAchine":
        clf.fit(X)
        y_pred = clf.predict(X)
        
    else:    
        clf.fit(X)
        scores_prediction = clf.decision_function(X)
        y_pred = clf.predict(X)
        
        
    #0 FOR VALID, 1 FOR FRAUDULENT
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1
    n_errors = (y_pred != Y).sum()
    
    #METRICS
    print("{}: {}".format(clf_name,n_errors))
    
    print("Accuracy Score :")
    
    print(accuracy_score(Y,y_pred))
    
    print("Classification Report :")
    
    print(classification_report(Y,y_pred))

* isolation forest was able to detect 675 errors compared to the 935 by local outlier factor. but if you look at the accuracy, isolation forest leads with 99.76% compared to local outlier which has 99.67%

* from this it is clear that isolation forest has a higher chance of detecting fraud which is at a great 31% while the latter detects fraud at 5% 

* The Isolation Forest method is the best way to detect fraud cases. It was/is able to detect the highest percentage of fraud in the creditcard dataset and with the least margin for error and in the future it can be used to detect fraud extensively through use of AI and deep learning algorithms powered by quantum computers to capture data in real time and relay it as a batch at the same time